## Introdução à Aprendizagem Profunda
**LISTA PRÁTICA DAS UNIDADES 1 E 2**
- Discente: Vitor Negromonte (vnco)
- Período: 2023.2

## Instruções e requisitos

- Pode ser feita com o grupo do projeto. Recomendo pair/group programming para que todos vejam um pouco de todas as partes.

- Treine e avalie 4 modelos de classificação para a base de dados do FashionMNIST (https://www.kaggle.com/datasets/zalando-research/fashionmnist,
https://pytorch.org/vision/stable/generated/torchvision.datasets.FashionMNIST.html).


1.   Um modelo base que não seja uma rede neural, como _decision tree, xgboost, random forest_, etc. Recomendação: use o sklearn (https://scikit-learn.org/).

2.   Uma MLP

3. Uma rede convolucional criada por ti. Recomendação: https://pytorch.org/

4. Use um modelo pré treinado já consolidado na literatura para fazer _transfer learning_. Recomendações: https://pytorch.org/hub/pytorch_vision_vgg/


Compare os resultados dos modelos:
* plote gráficos que mostrem as acurácias de cada modelo
* Indique qual foi a classe na qual o modelo teve pior performance (indique qual métrica usou para concluir isso e faça para cada modelo)
* argumente qual o melhor modelo levando em consideração o tempo de execução e acurácia.

Recomendação use:
https://pytorch.org/vision/main/generated/torchvision.datasets.MNIST.html .

Recomendação:

Faça um template de treino, validação e teste que funcione para uma API de modelo.

Crie a API para cada modelo que será usado e use o template

## Imports e Downloads

### Imports

In [15]:
import random
import time
import os
import pandas as pd
import numpy as np
import torch, torchvision
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
import seaborn as sns


### Downloading the dataset

In [5]:
# Load the FashionMNIST dataset
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True)

# Access the data and labels
train_data = train_dataset.data
train_labels = train_dataset.targets
test_data = test_dataset.data
test_labels = test_dataset.targets


100.0%


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100.0%


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100.0%


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100.0%


Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



### Functions

## Data preparation and Exploratory Data Analysis (EDA)

### Data preaparation

In [11]:
# Normalize the data
train_data = train_data.float()/255
test_data = test_data.float()/255

# Distribution of the labels in the traing dataset with numbers in each class
unique, counts = np.unique(train_labels, return_counts=True)
print(dict(zip(unique, counts)))


{0: 6000, 1: 6000, 2: 6000, 3: 6000, 4: 6000, 5: 6000, 6: 6000, 7: 6000, 8: 6000, 9: 6000}


### Exploratory data analysis

# Base Models

## Random Forest

## Suport Vector Machine

## Decision Tree

## KNN

# Multilayer Perceptron

# Convolutional network

# Transfer learning models

## VGG

## ResNet

## GoogLeNet

## AlexNet

# Comparative analysis